In [431]:
# Fonction defini au préalable
def segment_fundus(mask , fundus , manual_segments): 

    mask = cv2.imread(mask , 0 )
    manual_seg = cv2.imread(manual_segments  ,  0)
    dim = (565 , 584)
    manual_seg = cv2.resize(manual_seg, dim, interpolation = cv2.INTER_AREA)
    fundus = cv2.imread(fundus)
    b,green_fundus,r = cv2.split(fundus)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    contrast_enhanced_green_fundus = clahe.apply(green_fundus)

# applying alternate sequential filtering (3 times closing opening)
    r1 = cv2.morphologyEx(contrast_enhanced_green_fundus, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    R1 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    r2 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    R2 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    r3 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
    R3 = cv2.morphologyEx(r3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
    f4 = cv2.subtract(R3,contrast_enhanced_green_fundus)
    f5 = clahe.apply(f4)
    
    f5 = cv2.medianBlur(f5,5)

    #f5_close = cv2.morphologyEx(f5, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
    ret,f6 = cv2.threshold(f5,15,255,cv2.THRESH_BINARY)
    mask1 = np.ones(f5.shape[:2], dtype="uint8") * 255

    contours, im2 = cv2.findContours(f6.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) <= 200:
            cv2.drawContours(mask1, [cnt], -1, 0, -1)
            im = cv2.bitwise_and(f6, f6, mask=mask1)
            ret,fin = cv2.threshold(im,15,255,cv2.THRESH_BINARY_INV)
            newfin = cv2.erode(fin, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations=1)
    # removing blobs of unwanted bigger chunks taking in consideration they are not straight lines like blood
#vessels and also in an interval of area
    fundus_eroded = cv2.bitwise_not(newfin)
    xmask = np.ones(fundus.shape[:2], dtype="uint8") * 255
    xcontours, x1 = cv2.findContours(fundus_eroded.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in xcontours:
        shape = "unidentified"
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.04 * peri, False)
        if len(approx) > 4 and cv2.contourArea(cnt) <= 3000 and cv2.contourArea(cnt) >= 100:
            shape = "circle"
        else:
            shape = "veins"
        if(shape=="circle"):
            cv2.drawContours(xmask, [cnt], -1, 0, -1)
            finimage = cv2.bitwise_and(fundus_eroded,fundus_eroded,mask=xmask)
            blood_vessels = cv2.bitwise_not(finimage)

    blood_vessels_substracted = cv2.subtract(mask,blood_vessels)
    return blood_vessels_substracted

In [416]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity
import umetrics

In [432]:
Manual_seg_path = r"C:\Users\hp\Downloads\imageprocessingproject\retina-features-master\DRIVE\data\1st_manual\22_manual1.png"
masks_path = r'C:\Users\hp\Downloads\imageprocessingproject\retina-features-master\DRIVE\data\mask\mask22.png'
mask = cv2.imread(masks_path , 0 )
manual_seg = cv2.imread(Manual_seg_path  ,  0)
dim = (565 , 584)
manual_seg = cv2.resize(manual_seg, dim, interpolation = cv2.INTER_AREA)
print(mask.shape , manual_seg.shape)
PathFile = r"C:\Users\hp\Downloads\imageprocessingproject\retina-features-master\DRIVE\data\images\22_training.tif"
fundus = cv2.imread(PathFile)

(584, 565) (584, 565)


In [476]:
b,green_fundus,r = cv2.split(fundus)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
contrast_enhanced_green_fundus = clahe.apply(green_fundus)

# applying alternate sequential filtering (3 times closing opening)
o1 = cv2.morphologyEx(contrast_enhanced_green_fundus, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
c1 = cv2.morphologyEx(o1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
o2 = cv2.morphologyEx(c1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
c2 = cv2.morphologyEx(o2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
o3 = cv2.morphologyEx(c2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
c3 = cv2.morphologyEx(o3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
f4 = cv2.subtract(c3,contrast_enhanced_green_fundus)
f5 = clahe.apply(f4)

In [477]:
f5 = cv2.medianBlur(f5,5)
cv2.imshow("Processing1 image", f5)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [487]:
# removing very small contours through area parameter noise removal
ret,f6 = cv2.threshold(f5,20,255,cv2.THRESH_BINARY)
mask1 = np.ones(f5.shape[:2], dtype="uint8") * 255
cv2.imshow("Processing2 image", f6)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [488]:
contours, im2 = cv2.findContours(f6.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
for cnt in contours:
    if cv2.contourArea(cnt) <= 50:
        cv2.drawContours(mask1, [cnt], -1, 0, -1)
        im = cv2.bitwise_and(f6, f6, mask=mask1)
        ret,fin = cv2.threshold(im,15,255,cv2.THRESH_BINARY_INV)
        newfin = cv2.erode(fin, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations=1)
        
cv2.imshow('Noises Removed', newfin)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destro

In [486]:
# removing blobs of unwanted bigger chunks taking in consideration they are not straight lines like blood
#vessels and also in an interval of area
fundus_eroded = cv2.bitwise_not(newfin)
xmask = np.ones(fundus.shape[:2], dtype="uint8") * 255
xcontours, x1 = cv2.findContours(fundus_eroded.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
for cnt in xcontours:
    shape = "unidentified"
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.04 * peri, False)
    if len(approx) > 4 and cv2.contourArea(cnt) <= 3000 and cv2.contourArea(cnt) >= 100:
        shape = "circle"
    else:
        shape = "veins"
    if(shape=="circle"):
        cv2.drawContours(xmask, [cnt], -1, 0, -1)
        finimage = cv2.bitwise_and(fundus_eroded,fundus_eroded,mask=xmask)
        blood_vessels = cv2.bitwise_not(finimage)
cv2.imshow("Unwanted Shapes removed", blood_vessels)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
#eroding is done to eliminate the minor noises (post processing)

In [489]:
blood_vessels_substracted = cv2.subtract(mask,newfin)
cv2.imshow('Post-processings', blood_vessels_substracted)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destro

<h1> Evaluations  </h1>

In [490]:
# Evaluate Segmentation by comparing with the provided manual_segmentations

cv2.imshow('Predicted_Segments', blood_vessels_substracted)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image
cv2.imshow('Manual_segments', manual_seg)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image

In [491]:
sim , diff = structural_similarity(blood_vessels_substracted, manual_seg , full = True)
print(sim)

0.7622187199959611


In [492]:
cv2.imshow('Comparison', diff)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image

In [494]:
def orb_sim(img1 , img2) : 
    orb = cv2.ORB_create()
    kp_a , desc_a = orb.detectAndCompute(img1 , None)
    kp_b , desc_b = orb.detectAndCompute(img2 , None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING , crossCheck = True)
    # performing matches 
    matches = bf.match(desc_a , desc_b)
    #looking for similar regions with disntance < 50 
    similar_regions =[i for i in matches if i.distance < 50]
    if len(matches) == 0:
        return 0 
    return len(similar_regions) / len(matches)
orb_sim(blood_vessels_substracted , manual_seg)

0.7777777777777778

In [495]:
from sklearn.metrics import accuracy_score , confusion_matrix
accuracy_score(blood_vessels_substracted.reshape(-1) , manual_seg.reshape(-1))
matrice_de_confusion = confusion_matrix(blood_vessels_substracted.reshape(-1) , manual_seg.reshape(-1))

In [496]:
np.trace(matrice_de_confusion) / matrice_de_confusion.sum()

0.9503970178203418

In [497]:
import umetrics
result = umetrics.calculate(blood_vessels_substracted.reshape(-1), manual_seg.reshape(-1), strict=True, iou_threshold=0.5)

print(result.results)

 Segmentation Metrics (n=1)
Strict: True (IoU > 0.5)
n_true_labels: 2275
n_pred_labels: 6339
n_true_positives: 1724
n_false_positives: 622
n_false_negatives: 81
IoU: 0.784
Jaccard: 0.710
pixel_identity: 0.950
localization_error: 0.900



<h1> Inference </h1>

In [427]:
import os
MaskFolder = r"C:\Users\hp\Downloads\imageprocessingproject\retina-features-master\DRIVE\data\mask"
Manual_Segmentation_Folder = r"C:\Users\hp\Downloads\imageprocessingproject\retina-features-master\DRIVE\data\1st_manual"
pathFolder = r"C:\Users\hp\Downloads\imageprocessingproject\retina-features-master\DRIVE\data\images"
filesArray = [x for x in os.listdir(pathFolder) if os.path.isfile(os.path.join(pathFolder,x))]
MaskArray = [x for x in os.listdir(MaskFolder) if os.path.isfile(os.path.join(MaskFolder,x))]
ManualSegmentationArray = [x for x in os.listdir(Manual_Segmentation_Folder) if os.path.isfile(os.path.join(Manual_Segmentation_Folder,x))]

In [428]:
#Test segmentations
img = segment_fundus(masks_path , PathFile , Manual_seg_path)
cv2.imshow('Predicted_segmentation', img)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image

In [430]:
# Writing Predicted segmentation and comparing them with manual segmentations
destinationFolder = r"C:\Users\hp\Downloads\imageprocessingproject\retina-features-master\Segmented_Images"
comparisonFolder = r"C:\Users\hp\Downloads\imageprocessingproject\retina-features-master\ComparisonFolder"
if not os.path.exists(destinationFolder):
    os.mkdir(destinationFolder)
for file_name, mask_file , manualseg_file in  zip(filesArray , MaskArray , ManualSegmentationArray):
    file_name_no_extension = os.path.splitext(file_name)[0]
    mask = os.path.join(MaskFolder , mask_file)
    fundus = os.path.join(pathFolder , file_name)
    manual = os.path.join(Manual_Segmentation_Folder , manualseg_file)
    manual_image = cv2.imread(manual  ,  0)
    bloodvessel = segment_fundus(mask , fundus , manual)
    sim , diff = structural_similarity(bloodvessel, manual_image , full = True)
    cv2.imwrite(os.path.join(destinationFolder , file_name_no_extension) +  "_bloodvessel.png",bloodvessel)
    cv2.imwrite(os.path.join(comparisonFolder , file_name_no_extension) +  "comparison.png",diff*255)